<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/California_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
#

In [99]:
df = pd.read_csv('coliforniya_housing.csv')
df.head()

,Unnamed: 0,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,PRICE
0,0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


#### Eng kup takrorlangan mazilni markaz deb olamiz va markazdan qancha uzoqlikda ekanligini aniqlaymaz

In [100]:
df['Location'] = list(zip(df['Latitude'], df['Longitude']))

most_common_location = df['Location'].mode()[0]

center_latitude, center_longitude = most_common_location

def get_distance(row):
    house_coordinates = (row['Latitude'], row['Longitude'])
    return distance.euclidean(house_coordinates, (center_latitude, center_longitude))

df['distance'] = df.agg(get_distance, axis=1)
df.head()

,Unnamed: 0,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,PRICE,Location,distance
0,0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526,"(37.88, -122.23)",0.196977
1,1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585,"(37.86, -122.22)",0.199249
2,2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521,"(37.85, -122.24)",0.177200
3,3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413,"(37.85, -122.25)",0.167631
4,4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422,"(37.85, -122.25)",0.167631


#### KFold orqaliy model qilib kuramiz

In [105]:

X = df.drop(columns=['Unnamed: 0', 'PRICE',
                     'Location',
                     'Longitude',
                     'Latitude',
                     ])
y = df['PRICE']

model = LinearRegression()

kf = KFold(n_splits=6, shuffle=True, random_state=42)

scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kf)
rmse = -scores.mean()

model.fit(X, y)
y_pred = model.predict(X)

print(f"RMSE: {rmse}")

RMSE: 0.6172006650317786


PolynomialFeatures model qilib kuramiz

In [106]:
X = df.drop(columns=['Unnamed: 0', 'PRICE',
                     'Location',
                     'Longitude',
                     'Latitude',
                     ])
y = df['PRICE']

poly = PolynomialFeatures(degree=3, include_bias=False)
poly_x = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_x, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error')
rmse = -scores.mean()
print(f"RMSE: {rmse}")

model.fit(X, y)
y_pred = model.predict(X)

RMSE: 0.6516704505741121


#### Ma'lumotlarimizni Normalizatsiay qilib kuramiz

In [113]:
housing_data = df.drop(columns=['Unnamed: 0', 'Location', 'PRICE', 'Longitude', 'Location'])

scaler = MinMaxScaler(feature_range=(-1, 1))

normalized_data = scaler.fit_transform(housing_data)

X = pd.DataFrame(normalized_data, columns=housing_data.columns)
y = df['PRICE']

poly = PolynomialFeatures(degree=3, include_bias=False)
poly_x = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_x, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


kf = KFold(n_splits=3, shuffle=True, random_state=42)

scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kf)
rmse = -scores.mean()

print(f"RMSE: {rmse}")

RMSE: 0.5713027651269503


#### Bu dataset qatorlai kam bulganligi uchun PolynomialFeatures unchalik foyda bermadi.